## 4A. Práctica: Detección de Plagio
### Nombre: **Luis Fernando Izquierdo Berdugo**
### Materia: **Procesamiento de Información**
### Fecha: **28 de Octubre de 2024**



Instrucciones:

En esta actividad, el ejercicio propuesto es: **detectar a nivel básico, por medio de una medida de similitud, el plagio que hay dentro de un conjunto de documentos**.

Los datos son documentos de noticias, adaptados a la tarea de detección de plagio. En el conjunto de datos del directorio  `suspicious-documents`, hay  documentos que tienen fragmentos plagiados de los documentos fuente `source-documents`, sin embargo, también hay otros documentos que simularon plagio, pero no se plagió ningún párrafo de los documentos fuente.

La tarea es encontrar  el nivel de plagio entre los archivos del directorio `source-documents` y los archivos del directorio `suspicious-documents`.

Entregar un notebook con el código fuente para obtener la similitud entre dos documentos. Tomar 20 documentos de source-documents y por cada uno, encontrar los 5 archivos de suspicious-documents más parecidos. Indicar los nombres y la similitud obtenida.

Aplicar los preprocesamientos: usar solo letras minúsculas, remover stopwords (palabras que no son de contenido)  y aplicar un proceso de stemming para reducir el vocabulario.
 

Nota: Usar la lista de stopwords para eliminarlas, recordar si se le aplicó stemming, aplicar stemming también a las stopwords, debido a que alguna palabra podría cambiar.

1. Se recomienda usar el stemmer de NLTK. Para facilidad, usar Porter (inglés).

http://www.nltk.org/howto/stem.html

Básicamente usar:

from nltk.stem.porter import *

stemmer = PorterStemmer()

word = stemmer.stem('pages')

Word contendrá la palabra normalizada por medio de stemming ('page')  aplicar ese proceso a todos los textos. 

2. Para procesar la medida de similitud, usar la medida Jaccard y Dice

## Preprocesamiento
Lo primero que se hará será definitr la lista de stopwords y aplicarles stemming. Para esto se usará una instancia del Porter Stemmer de la biblioteca `nltk`.

In [15]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
# Descargar stopwords si no están descargadas
#nltk.download('stopwords')

# Crear una instancia del PorterStemmer
stemmer = PorterStemmer()

# Definir la lista de stopwords y aplicar stemming
stop_words = set(stopwords.words('english'))
stemmed_stop_words = {stemmer.stem(word) for word in stop_words}

Se definirá la función para leer los documentos. En esta se ordenan por nombre alfabético (para poder tomar los primeros 20 documentos) y se creará un diccionario con el nombre del archivo y el contenido de estos.

In [16]:
import os
def read_documents(directory, limit=None):
    documents = {}
    filenames = sorted([f for f in os.listdir(directory) if f.endswith(".txt")])
    for i, filename in enumerate(filenames):
        if limit is not None and i >= limit:
            break
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            documents[filename] = file.read()
    return documents

Lo siguiente será definir la función para el preprocesamiento del texto. En esta función se hará lo siguiente:
- Se convertirá a minúsculas el texto
- Se eliminarán caracteres especiales
- Se dividirá el texto en palabras
- Se aplicará el Porter stemming.
- Se eliminarán las stopwords

In [17]:
import re
def preprocess_text(text):
    """Preprocesa el texto: minúsculas, eliminar stopwords y aplicar stemming."""
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales
    text = re.sub(r'[^a-z\s]', '', text)
    # Dividir en palabras
    words = text.split()
    # Aplicar stemming y eliminar stopwords
    stemmed_words = [stemmer.stem(word) for word in words if word not in stemmed_stop_words]
    return ' '.join(stemmed_words)

Ya con las funciones se abren leen los archivos de las rutas y se preprocesa cada uno de ellos.

In [18]:
source_dir = "/Users/izluis/Documents/source-documents"
suspicious_dir = "/Users/izluis/Documents/suspicious-documents"

# Leer y preprocesar los documentos
source_documents = read_documents(source_dir, limit=20)
suspicious_documents = read_documents(suspicious_dir)

preprocessed_source_docs = {name: preprocess_text(text) for name, text in source_documents.items()}
preprocessed_suspicious_docs = {name: preprocess_text(text) for name, text in suspicious_documents.items()}


## Similitud de Jaccard

Se implementa con una función que efectúa lo siguiente:
- Divide los textos por palabras usando los espacios como delimitador
- Se convierte las listas de palabras en conjuntos para eliminar duplicados (así como permite que se hagan operaciones)
- Calcula la interseccion de los dos conjuntos (las palabras que están presentes en ambos textos)
- Calcula la unión de los dos conjuntos de palabras (todas las palabras que están mínimo en uno de los documentos)
- Se calcula la similitud de Jaccard dividiendo el tamaño de la intersección entre el tamaño de la unión.


In [19]:
def jaccard_similarity(doc1, doc2):
    words_doc1 = set(doc1.split())
    words_doc2 = set(doc2.split())
    intersection = words_doc1.intersection(words_doc2)
    union = words_doc1.union(words_doc2)
    return len(intersection) / len(union)

## Similitud de Dice

La función para calcular la similitud de Dice efectúa lo siguiente:
- Divide los textos por palabras usando los espacios como delimitador.
- Se converten las listas de palabras en conjuntos para eliminar duplicados (así como permite que se hagan operaciones)
- Se calcula la intersección de los dos conjuntos (las palabras que están presentes en ambos textos)
- Se calcula la similitud de Dice con la fórmula, esta dice que es el doble del tamaño de la intersección dividido entre la suma de los tamaños de los dos conjuntos

In [20]:
def dice_similarity(doc1, doc2):
    words_doc1 = set(doc1.split())
    words_doc2 = set(doc2.split())
    intersection = words_doc1.intersection(words_doc2)
    return 2 * len(intersection) / (len(words_doc1) + len(words_doc2))

## Cálculo de similutdes

Lo último será iterar por todos los `source documents` y encontrar los documentos más similares de `suspicious documents`. Primero se muestran los más similares por Jaccard y después los más similares por Dice.

In [23]:
# Calcular similitudes y encontrar los documentos más similares
results_jaccard = {}
results_dice = {}

for source_name, source_text in preprocessed_source_docs.items():
    similarities = []
    for suspicious_name, suspicious_text in preprocessed_suspicious_docs.items():
        jaccard_sim = jaccard_similarity(source_text, suspicious_text)
        dice_sim = dice_similarity(source_text, suspicious_text)
        similarities.append((suspicious_name, jaccard_sim, dice_sim))
    
    # Ordenar por similitud de Jaccard y tomar los 5 más similares
    similarities.sort(key=lambda x: x[1], reverse=True)
    results_jaccard[source_name] = similarities[:5]
    
    # Ordenar por similitud de Dice y tomar los 5 más similares
    similarities.sort(key=lambda x: x[2], reverse=True)
    results_dice[source_name] = similarities[:5]

# Mostrar resultados
print("Resultados por Jaccard:")
for source_name, similar_docs in results_jaccard.items():
    print(f"Documentos más similares a {source_name}:")
    for doc_name, jaccard_sim, dice_sim in similar_docs:
        print(f"  {doc_name}: Jaccard={jaccard_sim:.4f}")
    print()

print("-----------------------------------------------")
print("Resultados por Dice:")
for source_name, similar_docs in results_dice.items():
    print(f"Documentos más similares a {source_name}:")
    for doc_name, jaccard_sim, dice_sim in similar_docs:
        print(f"  {doc_name}: Dice={dice_sim:.4f}")
    print()

Resultados por Jaccard:
Documentos más similares a source-document0001.txt:
  suspicious-document0010.txt: Jaccard=0.1820
  suspicious-document2205.txt: Jaccard=0.1774
  suspicious-document2048.txt: Jaccard=0.1762
  suspicious-document2121.txt: Jaccard=0.1756
  suspicious-document1267.txt: Jaccard=0.1736

Documentos más similares a source-document0002.txt:
  suspicious-document0020.txt: Jaccard=0.1449
  suspicious-document0019.txt: Jaccard=0.1278
  suspicious-document1463.txt: Jaccard=0.1164
  suspicious-document0013.txt: Jaccard=0.1143
  suspicious-document1603.txt: Jaccard=0.1132

Documentos más similares a source-document0003.txt:
  suspicious-document0030.txt: Jaccard=0.2004
  suspicious-document0029.txt: Jaccard=0.1734
  suspicious-document1000.txt: Jaccard=0.1123
  suspicious-document0023.txt: Jaccard=0.1098
  suspicious-document0022.txt: Jaccard=0.1080

Documentos más similares a source-document0004.txt:
  suspicious-document0040.txt: Jaccard=0.1782
  suspicious-document0039.txt